# Value of on-ice events

<p>data frames used in this notebook:</p>
<p>&nbsp; &nbsp; 1. all on-ice prior to a goal events.</p>
<p>&nbsp; &nbsp; 2. all even strength on-ice events.</p> 
 

## 1) all on-ice events prior to a goal

In [83]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy
import zipfile


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

In [84]:
pwd

'/Users/stefanostselios/Desktop/nhl_roster_design-master'

In [85]:
da = pd.read_csv('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/pbp_merged.csv')
#da = pd.read_csv('/Users/kevinmongeon/Brock University/Steve Tselios - StephanosShare/out/pbp_merged.csv')
da = da.drop('Unnamed: 0', axis=1)
da = da.rename(columns={'TeamCode': 'EventTeamCode'})

- keep regular season games and relevant on-ice events in **regulation time**. Drop duplicates by season, game number, event number and event team to have one obsrevation per event per game.

In [86]:
da = da[da['GameNumber'] <= 21230]
da = da[da['Period'] <= 3]
da = da[da['Period'] >= 1]
da = da[da['EventType']!='STOP']
da = da[da['EventType']!='EISTR']
da = da[da['EventType']!='EIEND']
da = da[da['EventType'] !='FIGHT']
da = da.dropna(subset=['EventNumber'])

In [87]:
da.head()

,Season,GameNumber,EventNumber,Period,AdvantageType,EventTimeFromZero,EventTimeFromTwenty,EventType,EventDetail,VPlayer1,VPosition1,VPlayer2,VPosition2,VPlayer3,VPosition3,VPlayer4,VPosition4,VPlayer5,VPosition5,VPlayer6,VPosition6,HPlayer1,HPosition1,HPlayer2,HPosition2,HPlayer3,HPosition3,HPlayer4,HPosition4,HPlayer5,HPosition5,HPlayer6,HPosition6,GameDate,VTeamCode,HTeamCode,EventTeamCode,PlayerNumber,PlayerName,ShotType,ShotResult,Zone,Length,PenaltyType
0,2010,20001,1,1,NaN,0,1200,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,11,C,21.0,R,57.0,L,26.0,D,75.0,D,31.0,G,37,C,9.0,R,11.0,L,3.0,D,22.0,D,35.0,G,2010-10-07,MTL,TOR,MTL,11.0,GOMEZ,NaN,NaN,N,NaN,NaN
1,2010,20001,3,1,EV,15,1185,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",11,C,21.0,R,57.0,L,26.0,D,75.0,D,31.0,G,37,C,9.0,R,11.0,L,3.0,D,22.0,D,35.0,G,2010-10-07,MTL,TOR,TOR,37.0,BRENT,NaN,NaN,O,NaN,NaN
2,2010,20001,4,1,EV,46,1154,HIT,"MTL #14 PLEKANEC HIT TOR #2 SCHENN, Off. Zone",14,C,81.0,C,46.0,L,6.0,D,76.0,D,31.0,G,42,C,81.0,C,32.0,R,2.0,D,15.0,D,35.0,G,2010-10-07,MTL,TOR,MTL,14.0,PLEKANEC,NaN,NaN,O,NaN,NaN
3,2010,20001,5,1,EV,57,1143,HIT,"MTL #76 SUBBAN HIT TOR #15 KABERLE, Neu. Zone",14,C,81.0,C,46.0,L,6.0,D,76.0,D,31.0,G,42,C,81.0,C,32.0,R,2.0,D,15.0,D,35.0,G,2010-10-07,MTL,TOR,MTL,76.0,SUBBAN,NaN,NaN,N,NaN,NaN
4,2010,20001,6,1,EV,69,1131,GIVE,"TOR&nbsp;GIVEAWAY - #35 GIGUERE, Def. Zone",14,C,81.0,C,46.0,L,6.0,D,76.0,D,31.0,G,42,C,81.0,C,32.0,R,2.0,D,15.0,D,35.0,G,2010-10-07,MTL,TOR,TOR,35.0,GIGUERE,NaN,NaN,D,NaN,NaN


In [88]:
da.shape

(310113, 44)

- create a goal dataframe that will display the number of goal per game.

In [89]:
df = da[['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'VTeamCode', 'HTeamCode', 'EventTeamCode']]
dg = df[df['EventType'] == 'GOAL']
dg['Goal'] = dg.apply(lambda x: 1 if (x['EventType'] == 'GOAL') else 0, axis=1)
dg['GoalNumber'] = dg.groupby(['Season', 'GameNumber']).cumcount()+1
dg.head()
dg = dg[['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'EventTeamCode', 'VTeamCode', 'HTeamCode', 'GoalNumber']]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


- merge dg onto df to display the goal number per game. Group by season, game number and period to backwardfill advantage type and goal number.

In [90]:
df = pd.merge(df, dg, on=['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'EventTeamCode', 'VTeamCode', 'HTeamCode'], how='left')
df['AdvantageType'] = df.groupby(['Season', 'GameNumber'])['AdvantageType'].apply(lambda x: x.bfill())
df['GoalNumber'] = df.groupby(['Season', 'GameNumber', 'Period'])['GoalNumber'].apply(lambda x: x.bfill())
df.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,VTeamCode,HTeamCode,EventTeamCode,GoalNumber
0,2010,20001,1,EV,1,FAC,0,MTL,TOR,MTL,1.0
1,2010,20001,3,EV,1,HIT,15,MTL,TOR,TOR,1.0
2,2010,20001,4,EV,1,HIT,46,MTL,TOR,MTL,1.0
3,2010,20001,5,EV,1,HIT,57,MTL,TOR,MTL,1.0
4,2010,20001,6,EV,1,GIVE,69,MTL,TOR,TOR,1.0


##  all on-ice events prior to a goal

- display the home goal number and visitor goal number by game number and season. Keep all on-ice events that happened prior to a goal when the score differential was between -1 and 1. Exclude all other events.

In [91]:
dz = dg[dg['EventTeamCode'] == dg['HTeamCode']]
dz['HGoalNumber'] = dz.groupby(['Season', 'GameNumber']).cumcount()+1
dy = dg[dg['EventTeamCode'] == dg['VTeamCode']]
dy['VGoalNumber'] = dy.groupby(['Season', 'GameNumber']).cumcount()+1

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


- merge visitor goal number dataframe (dy) and home goal number dataframe (dz) onto goal dataframe (dg). 

In [92]:
dg = pd.merge(dg, dy, on=['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'EventTeamCode', 'VTeamCode', 'HTeamCode', 'GoalNumber'], how='left')
dg = pd.merge(dg, dz, on=['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'EventTeamCode', 'VTeamCode', 'HTeamCode', 'GoalNumber'], how='left')
dg.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,EventTeamCode,VTeamCode,HTeamCode,GoalNumber,VGoalNumber,HGoalNumber
0,2010,20001,35,EV,1,GOAL,402,TOR,MTL,TOR,1,NaN,1.0
1,2010,20001,49,EV,1,GOAL,537,TOR,MTL,TOR,2,NaN,2.0
2,2010,20001,68,EV,1,GOAL,739,MTL,MTL,TOR,3,1.0,NaN
3,2010,20001,223,EV,3,GOAL,96,TOR,MTL,TOR,4,NaN,3.0
4,2010,20001,232,EV,3,GOAL,148,MTL,MTL,TOR,5,2.0,NaN


- forward fill home goal number and visitor game number by season and game number. Fill in 'NaN' values with zero for home and visitor game number.

In [93]:
dg['HGoalNumber'] = dg.groupby(['Season', 'GameNumber'])['HGoalNumber'].apply(lambda x: x.ffill())
dg['VGoalNumber'] = dg.groupby(['Season', 'GameNumber'])['VGoalNumber'].apply(lambda x: x.ffill())
dg['VGoalNumber'] = dg['VGoalNumber'].fillna(0)
dg['HGoalNumber'] = dg['HGoalNumber'].fillna(0)
dg.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,EventTeamCode,VTeamCode,HTeamCode,GoalNumber,VGoalNumber,HGoalNumber
0,2010,20001,35,EV,1,GOAL,402,TOR,MTL,TOR,1,0.0,1.0
1,2010,20001,49,EV,1,GOAL,537,TOR,MTL,TOR,2,0.0,2.0
2,2010,20001,68,EV,1,GOAL,739,MTL,MTL,TOR,3,1.0,2.0
3,2010,20001,223,EV,3,GOAL,96,TOR,MTL,TOR,4,1.0,3.0
4,2010,20001,232,EV,3,GOAL,148,MTL,MTL,TOR,5,2.0,3.0


- merge goal dataframe on dk and backward fill by home goal number and visitor goal number.

In [94]:
dk = da[['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'VTeamCode', 'HTeamCode', 'EventTeamCode']]
dk = pd.merge(dk, dg, on=['Season', 'GameNumber', 'EventNumber', 'AdvantageType', 'Period', 'EventType', 'EventTimeFromZero', 'EventTeamCode', 'VTeamCode', 'HTeamCode'], how='left')
dk['AdvantageType'] = dk.groupby(['Season', 'GameNumber'])['AdvantageType'].apply(lambda x: x.bfill())
dk['GoalNumber'] = dk.groupby(['Season', 'GameNumber', 'Period'])['GoalNumber'].apply(lambda x: x.bfill())
dk['HGoalNumber'] = dk.groupby(['Season', 'GameNumber', 'Period'])['HGoalNumber'].apply(lambda x: x.bfill())
dk['VGoalNumber'] = dk.groupby(['Season', 'GameNumber', 'Period'])['VGoalNumber'].apply(lambda x: x.bfill())
dk = dk[dk['AdvantageType'] == 'EV']
dk.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,VTeamCode,HTeamCode,EventTeamCode,GoalNumber,VGoalNumber,HGoalNumber
0,2010,20001,1,EV,1,FAC,0,MTL,TOR,MTL,1.0,0.0,1.0
1,2010,20001,3,EV,1,HIT,15,MTL,TOR,TOR,1.0,0.0,1.0
2,2010,20001,4,EV,1,HIT,46,MTL,TOR,MTL,1.0,0.0,1.0
3,2010,20001,5,EV,1,HIT,57,MTL,TOR,MTL,1.0,0.0,1.0
4,2010,20001,6,EV,1,GIVE,69,MTL,TOR,TOR,1.0,0.0,1.0


- display the goal differential per game for each team.

In [95]:
dk['GD'] = dk.apply(lambda x: x['HGoalNumber'] - x['VGoalNumber'] if (x['EventTeamCode'] == x['HTeamCode']) else x['VGoalNumber'] - x['HGoalNumber'], axis=1)
dk.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,VTeamCode,HTeamCode,EventTeamCode,GoalNumber,VGoalNumber,HGoalNumber,GD
0,2010,20001,1,EV,1,FAC,0,MTL,TOR,MTL,1.0,0.0,1.0,-1.0
1,2010,20001,3,EV,1,HIT,15,MTL,TOR,TOR,1.0,0.0,1.0,1.0
2,2010,20001,4,EV,1,HIT,46,MTL,TOR,MTL,1.0,0.0,1.0,-1.0
3,2010,20001,5,EV,1,HIT,57,MTL,TOR,MTL,1.0,0.0,1.0,-1.0
4,2010,20001,6,EV,1,GIVE,69,MTL,TOR,TOR,1.0,0.0,1.0,1.0


In [96]:
dk.shape

(256153, 14)

- On-ice events that occured in a different period from a goal or after a goal are excluded from the dataframe.

In [97]:
dk = dk.dropna(subset=['GoalNumber'])
dk = dk.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dk = dk.drop_duplicates(['Season', 'GameNumber', 'EventNumber', 'EventTeamCode'])
dk.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,VTeamCode,HTeamCode,EventTeamCode,GoalNumber,VGoalNumber,HGoalNumber,GD
0,2010,20001,1,EV,1,FAC,0,MTL,TOR,MTL,1.0,0.0,1.0,-1.0
1,2010,20001,3,EV,1,HIT,15,MTL,TOR,TOR,1.0,0.0,1.0,1.0
2,2010,20001,4,EV,1,HIT,46,MTL,TOR,MTL,1.0,0.0,1.0,-1.0
3,2010,20001,5,EV,1,HIT,57,MTL,TOR,MTL,1.0,0.0,1.0,-1.0
4,2010,20001,6,EV,1,GIVE,69,MTL,TOR,TOR,1.0,0.0,1.0,1.0


In [98]:
dk.shape

(147273, 14)

- Assign a value of 1 if an on-ice event is a goal, 0 if not. Follow the same procedure for block, faceoff, giveaway, hits, miss, penalty, shot and takeaway. Group by season, game number and event type to find the sum of each on-ice event per game. 

In [99]:
dk['Goal'] = dk.apply(lambda x: 1 if (x['EventType'] == 'GOAL') else np.nan, axis=1)
dk['Block'] = dk.apply(lambda x: 1 if (x['EventType'] == 'BLOCK') else np.nan, axis=1)
dk['Faceoff'] = dk.apply(lambda x: 1 if (x['EventType'] == 'FAC') else np.nan, axis=1)
dk['Giveaway'] = dk.apply(lambda x: 1 if (x['EventType'] == 'GIVE') else np.nan, axis=1)
dk['Hit'] = dk.apply(lambda x: 1 if (x['EventType'] == 'HIT') else np.nan, axis=1)
dk['Miss'] = dk.apply(lambda x: 1 if (x['EventType'] == 'MISS') else np.nan, axis=1)
dk['Penalty'] = dk.apply(lambda x: 1 if (x['EventType'] == 'PENL') else np.nan, axis=1)
dk['Shot'] = dk.apply(lambda x: 1 if (x['EventType'] == 'SHOT') else np.nan, axis=1)
dk['Takeaway'] = dk.apply(lambda x: 1 if (x['EventType'] == 'TAKE') else np.nan, axis=1)

In [100]:
dk['Blocks'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Block'].transform('sum')
dk['Faceoffs'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Faceoff'].transform('sum')
dk['Giveaways'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Giveaway'].transform('sum')
dk['Goals'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Goal'].transform('sum')
dk['Hits'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Hit'].transform('sum')
dk['Misses'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Miss'].transform('sum')
dk['Penalties'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Penalty'].transform('sum')
dk['Shots'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Shot'].transform('sum')
dk['Takeaways'] = dk.groupby(['Season','GameNumber', 'EventTeamCode', 'EventType', 'GoalNumber'])['Takeaway'].transform('sum')

In [101]:
dk.head()

,Season,GameNumber,EventNumber,AdvantageType,Period,EventType,EventTimeFromZero,VTeamCode,HTeamCode,EventTeamCode,GoalNumber,VGoalNumber,HGoalNumber,GD,Goal,Block,Faceoff,Giveaway,Hit,Miss,Penalty,Shot,Takeaway,Blocks,Faceoffs,Giveaways,Goals,Hits,Misses,Penalties,Shots,Takeaways
0,2010,20001,1,EV,1,FAC,0,MTL,TOR,MTL,1.0,0.0,1.0,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,20001,3,EV,1,HIT,15,MTL,TOR,TOR,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
2,2010,20001,4,EV,1,HIT,46,MTL,TOR,MTL,1.0,0.0,1.0,-1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
3,2010,20001,5,EV,1,HIT,57,MTL,TOR,MTL,1.0,0.0,1.0,-1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
4,2010,20001,6,EV,1,GIVE,69,MTL,TOR,TOR,1.0,0.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
dk.shape

(147273, 32)

- reshape data wide to long.

In [103]:
dk = dk.rename(columns={'EventTeamCode': 'EventTeam'})
a = [col for col in dk.columns if 'TeamCode' in col]
dk = pd.lreshape(dk, {'TeamCode' : a})
dk = dk.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dk = dk.rename(columns={'EventTeam': 'EventTeamCode'})
dk.head()

,AdvantageType,Block,Blocks,EventNumber,EventTeamCode,EventTimeFromZero,EventType,Faceoff,Faceoffs,GD,GameNumber,Giveaway,Giveaways,Goal,GoalNumber,Goals,HGoalNumber,Hit,Hits,Miss,Misses,Penalties,Penalty,Period,Season,Shot,Shots,Takeaway,Takeaways,VGoalNumber,TeamCode
0,EV,NaN,NaN,1,MTL,0,FAC,1.0,2.0,-1.0,20001,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,2010,NaN,NaN,NaN,NaN,0.0,MTL
147273,EV,NaN,NaN,1,MTL,0,FAC,1.0,2.0,-1.0,20001,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,2010,NaN,NaN,NaN,NaN,0.0,TOR
1,EV,NaN,NaN,3,TOR,15,HIT,NaN,NaN,1.0,20001,NaN,NaN,NaN,1.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1,2010,NaN,NaN,NaN,NaN,0.0,MTL
147274,EV,NaN,NaN,3,TOR,15,HIT,NaN,NaN,1.0,20001,NaN,NaN,NaN,1.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1,2010,NaN,NaN,NaN,NaN,0.0,TOR
2,EV,NaN,NaN,4,MTL,46,HIT,NaN,NaN,-1.0,20001,NaN,NaN,NaN,1.0,NaN,1.0,1.0,7.0,NaN,NaN,NaN,NaN,1,2010,NaN,NaN,NaN,NaN,0.0,MTL


- drop duplicates by season, game number, team code and event type.

In [104]:
dk = dk.drop_duplicates(['Season', 'GameNumber', 'TeamCode', 'EventTeamCode', 'EventType', 'GoalNumber'])
dk = dk [['Season', 'GameNumber', 'AdvantageType', 'Period', 'TeamCode', 'EventNumber', 'EventType', 'EventTeamCode', 'GoalNumber', 'GD',  'Blocks', 'Faceoffs', 'Giveaways', 'Goals', 'Hits', 'Misses', 'Penalties', 'Shots', 'Takeaways']]
dk = dk.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dk.shape

(125428, 19)

- assign all on-ice events to their respectful teams. If team code is the same as event team code, then the on-ice event is assigned to that team. If not it is assigned to the opposing team. Each on-ice event generates two variables per team: For (F) and Against (A).

In [105]:
dk['Blocks_F'] = dk.apply(lambda x: x['Blocks'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Blocks_A'] = dk.apply(lambda x: x['Blocks'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Faceoffs_F'] = dk.apply(lambda x: x['Faceoffs'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Faceoffs_A'] = dk.apply(lambda x: x['Faceoffs'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Giveaways_F'] = dk.apply(lambda x: x['Giveaways'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Giveaways_A'] = dk.apply(lambda x: x['Giveaways'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Goals_F'] = dk.apply(lambda x: x['Goals'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Goals_A'] = dk.apply(lambda x: x['Goals'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Hits_F'] = dk.apply(lambda x: x['Hits'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Hits_A'] = dk.apply(lambda x: x['Hits'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Miss_F'] = dk.apply(lambda x: x['Misses'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Miss_A'] = dk.apply(lambda x: x['Misses'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Penalties_F'] = dk.apply(lambda x: x['Penalties'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Penalties_A'] = dk.apply(lambda x: x['Penalties'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Shots_F'] = dk.apply(lambda x: x['Shots'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Shots_A'] = dk.apply(lambda x: x['Shots'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk['Takeaways_F'] = dk.apply(lambda x: x['Takeaways'] if (x['TeamCode'] == x['EventTeamCode']) else np.nan, axis=1)
dk['Takeaways_A'] = dk.apply(lambda x: x['Takeaways'] if (x['TeamCode'] != x['EventTeamCode']) else np.nan, axis=1)
dk = dk.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dk.head()

,Season,GameNumber,AdvantageType,Period,TeamCode,EventNumber,EventType,EventTeamCode,GoalNumber,GD,Blocks,Faceoffs,Giveaways,Goals,Hits,Misses,Penalties,Shots,Takeaways,Blocks_F,Blocks_A,Faceoffs_F,Faceoffs_A,Giveaways_F,Giveaways_A,Goals_F,Goals_A,Hits_F,Hits_A,Miss_F,Miss_A,Penalties_F,Penalties_A,Shots_F,Shots_A,Takeaways_F,Takeaways_A
0,2010,20001,EV,1,MTL,1,FAC,MTL,1.0,-1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147273,2010,20001,EV,1,TOR,1,FAC,MTL,1.0,-1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,20001,EV,1,MTL,3,HIT,TOR,1.0,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147274,2010,20001,EV,1,TOR,3,HIT,TOR,1.0,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,20001,EV,1,MTL,4,HIT,MTL,1.0,-1.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- backward and forward fill of on-ice events by season, game number and team code.

In [106]:
dk['Blocks_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Blocks_F'].apply(lambda x: x.ffill().bfill())
dk['Faceoffs_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Faceoffs_F'].apply(lambda x: x.ffill().bfill())
dk['Giveaways_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Giveaways_F'].apply(lambda x: x.ffill().bfill())
dk['Goals_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Goals_F'].apply(lambda x: x.ffill().bfill())
dk['Hits_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Hits_F'].apply(lambda x: x.ffill().bfill())
dk['Miss_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Miss_F'].apply(lambda x: x.ffill().bfill())
dk['Penalties_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Penalties_F'].apply(lambda x: x.ffill().bfill())
dk['Shots_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Shots_F'].apply(lambda x: x.ffill().bfill())
dk['Takeaways_F'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Takeaways_F'].apply(lambda x: x.ffill().bfill())
dk['Blocks_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Blocks_A'].apply(lambda x: x.ffill().bfill())
dk['Faceoffs_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Faceoffs_A'].apply(lambda x: x.ffill().bfill())
dk['Giveaways_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Giveaways_A'].apply(lambda x: x.ffill().bfill())
dk['Goals_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Goals_A'].apply(lambda x: x.ffill().bfill())
dk['Hits_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Hits_A'].apply(lambda x: x.ffill().bfill())
dk['Miss_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Miss_A'].apply(lambda x: x.ffill().bfill())
dk['Penalties_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Penalties_A'].apply(lambda x: x.ffill().bfill())
dk['Shots_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Shots_A'].apply(lambda x: x.ffill().bfill())
dk['Takeaways_A'] = dk.groupby(['Season','GameNumber', 'TeamCode', 'GoalNumber'])['Takeaways_A'].apply(lambda x: x.ffill().bfill())
dk = dk.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dk = dk.fillna(0)
dk.head()

,Season,GameNumber,AdvantageType,Period,TeamCode,EventNumber,EventType,EventTeamCode,GoalNumber,GD,Blocks,Faceoffs,Giveaways,Goals,Hits,Misses,Penalties,Shots,Takeaways,Blocks_F,Blocks_A,Faceoffs_F,Faceoffs_A,Giveaways_F,Giveaways_A,Goals_F,Goals_A,Hits_F,Hits_A,Miss_F,Miss_A,Penalties_F,Penalties_A,Shots_F,Shots_A,Takeaways_F,Takeaways_A
0,2010,20001,EV,1,MTL,1,FAC,MTL,1.0,-1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
147273,2010,20001,EV,1,TOR,1,FAC,MTL,1.0,-1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,1.0,2.0,2.0,3.0,1.0,0.0,3.0,7.0,0.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0
1,2010,20001,EV,1,MTL,3,HIT,TOR,1.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
147274,2010,20001,EV,1,TOR,3,HIT,TOR,1.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,1.0,2.0,2.0,3.0,1.0,0.0,3.0,7.0,0.0,1.0,0.0,0.0,3.0,2.0,0.0,0.0
2,2010,20001,EV,1,MTL,4,HIT,MTL,1.0,-1.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0


- keep only relative columns and drop duplicates by season, gamenumber and teamcode, to have two observations per game.

In [107]:
dk = dk[['Season', 'GameNumber', 'EventTeamCode', 'GoalNumber', 'GD', 'Blocks_F', 'Blocks_A', 'Faceoffs_F', 'Faceoffs_A', 'Giveaways_F', 'Giveaways_A', 'Goals_F', 'Goals_A', 'Hits_F', 'Hits_A', 'Miss_F', 'Miss_A', 'Penalties_F', 'Penalties_A', 'Shots_F', 'Shots_A', 'Takeaways_F', 'Takeaways_A']]
dk = dk.sort_values(['Season', 'GameNumber'], ascending=[True, True])
dk = dk.drop_duplicates(['Season', 'GameNumber', 'EventTeamCode', 'GoalNumber', 'GD'])
dk = dk.rename(columns={'EventTeamCode': 'TeamCode'})
dk.head()

,Season,GameNumber,TeamCode,GoalNumber,GD,Blocks_F,Blocks_A,Faceoffs_F,Faceoffs_A,Giveaways_F,Giveaways_A,Goals_F,Goals_A,Hits_F,Hits_A,Miss_F,Miss_A,Penalties_F,Penalties_A,Shots_F,Shots_A,Takeaways_F,Takeaways_A
0,2010,20001,MTL,1.0,-1.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
1,2010,20001,TOR,1.0,1.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
32,2010,20001,MTL,2.0,-2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
34,2010,20001,TOR,2.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
44,2010,20001,MTL,3.0,-1.0,1.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,4.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0


In [108]:
dk.shape

(12660, 23)

In [109]:
dk.isnull().sum()

Season         0
GameNumber     0
TeamCode       0
GoalNumber     0
GD             0
Blocks_F       0
Blocks_A       0
Faceoffs_F     0
Faceoffs_A     0
Giveaways_F    0
Giveaways_A    0
Goals_F        0
Goals_A        0
Hits_F         0
Hits_A         0
Miss_F         0
Miss_A         0
Penalties_F    0
Penalties_A    0
Shots_F        0
Shots_A        0
Takeaways_F    0
Takeaways_A    0
dtype: int64

- group by season, team code and goal differential to compute the mean of each on-ice events while score differential was the same throughout the season.

In [110]:
dk['MBlocks_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Blocks_F'].transform('mean')
dk['MFaceoffs_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Faceoffs_F'].transform('mean')
dk['MGiveaways_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Giveaways_F'].transform('mean')
dk['MGoals_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Goals_F'].transform('mean')
dk['MHits_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Hits_F'].transform('mean')
dk['MMiss_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Miss_F'].transform('mean')
dk['MPenalties_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Penalties_F'].transform('mean')
dk['MShots_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Shots_F'].transform('mean')
dk['MTakeaways_F'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Takeaways_F'].transform('mean')
dk['MBlocks_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Blocks_A'].transform('mean')
dk['MFaceoffs_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Faceoffs_A'].transform('mean')
dk['MGiveaways_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Giveaways_A'].transform('mean')
dk['MGoals_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Goals_A'].transform('mean')
dk['MHits_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Hits_A'].transform('mean')
dk['MMiss_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Miss_A'].transform('mean')
dk['MPenalties_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Penalties_A'].transform('mean')
dk['MShots_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Shots_A'].transform('mean')
dk['MTakeaways_A'] = dk.groupby(['Season', 'TeamCode', 'GD'])['Takeaways_A'].transform('mean')
dk.head()

,Season,GameNumber,TeamCode,GoalNumber,GD,Blocks_F,Blocks_A,Faceoffs_F,Faceoffs_A,Giveaways_F,Giveaways_A,Goals_F,Goals_A,Hits_F,Hits_A,Miss_F,Miss_A,Penalties_F,Penalties_A,Shots_F,Shots_A,Takeaways_F,Takeaways_A,MBlocks_F,MFaceoffs_F,MGiveaways_F,MGoals_F,MHits_F,MMiss_F,MPenalties_F,MShots_F,MTakeaways_F,MBlocks_A,MFaceoffs_A,MGiveaways_A,MGoals_A,MHits_A,MMiss_A,MPenalties_A,MShots_A,MTakeaways_A
0,2010,20001,MTL,1.0,-1.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,1.784810,2.822785,1.088608,0.367089,2.784810,1.291139,0.544304,2.696203,0.544304,1.582278,2.810127,1.240506,0.443038,3.113924,1.481013,0.620253,3.202532,1.050633
1,2010,20001,TOR,1.0,1.0,3.0,4.0,2.0,1.0,3.0,2.0,0.0,1.0,7.0,3.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,1.488372,2.244186,0.837209,0.383721,2.476744,1.034884,0.430233,2.244186,0.604651,1.476744,2.441860,1.209302,0.406977,2.383721,1.151163,0.337209,2.279070,0.697674
32,2010,20001,MTL,2.0,-2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.437500,3.166667,0.895833,0.375000,2.562500,1.041667,0.562500,2.937500,0.375000,1.416667,3.020833,1.166667,0.395833,2.812500,1.125000,0.750000,2.666667,0.791667
34,2010,20001,TOR,2.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.333333,2.357143,0.952381,0.428571,2.309524,1.190476,0.476190,2.238095,0.571429,1.880952,2.642857,1.285714,0.452381,2.500000,1.214286,0.523810,2.595238,1.023810
44,2010,20001,MTL,3.0,-1.0,1.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,4.0,4.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.784810,2.822785,1.088608,0.367089,2.784810,1.291139,0.544304,2.696203,0.544304,1.582278,2.810127,1.240506,0.443038,3.113924,1.481013,0.620253,3.202532,1.050633


- drop duplicates by season, team code and goal differential.

In [111]:
dk = dk.drop_duplicates(['Season', 'TeamCode', 'GD'])
dk = dk [['Season', 'TeamCode', 'GD','MBlocks_F', 'MFaceoffs_F', 'MGiveaways_F', 'MGoals_F','MHits_F', 'MMiss_F', 'MPenalties_F', 'MShots_F', 'MTakeaways_F','MBlocks_A', 'MFaceoffs_A', 'MGiveaways_A', 'MGoals_A', 'MHits_A','MMiss_A', 'MPenalties_A', 'MShots_A', 'MTakeaways_A']]
dk.head()

,Season,TeamCode,GD,MBlocks_F,MFaceoffs_F,MGiveaways_F,MGoals_F,MHits_F,MMiss_F,MPenalties_F,MShots_F,MTakeaways_F,MBlocks_A,MFaceoffs_A,MGiveaways_A,MGoals_A,MHits_A,MMiss_A,MPenalties_A,MShots_A,MTakeaways_A
0,2010,MTL,-1.0,1.784810,2.822785,1.088608,0.367089,2.784810,1.291139,0.544304,2.696203,0.544304,1.582278,2.810127,1.240506,0.443038,3.113924,1.481013,0.620253,3.202532,1.050633
1,2010,TOR,1.0,1.488372,2.244186,0.837209,0.383721,2.476744,1.034884,0.430233,2.244186,0.604651,1.476744,2.441860,1.209302,0.406977,2.383721,1.151163,0.337209,2.279070,0.697674
32,2010,MTL,-2.0,1.437500,3.166667,0.895833,0.375000,2.562500,1.041667,0.562500,2.937500,0.375000,1.416667,3.020833,1.166667,0.395833,2.812500,1.125000,0.750000,2.666667,0.791667
34,2010,TOR,2.0,1.333333,2.357143,0.952381,0.428571,2.309524,1.190476,0.476190,2.238095,0.571429,1.880952,2.642857,1.285714,0.452381,2.500000,1.214286,0.523810,2.595238,1.023810
77,2010,PIT,-1.0,1.202899,2.594203,0.449275,0.347826,3.000000,0.855072,0.507246,2.507246,0.492754,1.347826,2.521739,0.666667,0.463768,3.333333,0.956522,0.449275,2.579710,0.449275


### summary analysis

In [112]:
dk['TBlocks_F'] = dk.groupby(['Season', 'GD'])['MBlocks_F'].transform('mean')
dk['TFaceoffs_F'] = dk.groupby(['Season', 'GD'])['MFaceoffs_F'].transform('mean')
dk['TGiveaways_F'] = dk.groupby(['Season', 'GD'])['MGiveaways_F'].transform('mean')
dk['TGoals_F'] = dk.groupby(['Season', 'GD'])['MGoals_F'].transform('mean')
dk['THits_F'] = dk.groupby(['Season', 'GD'])['MHits_F'].transform('mean')
dk['TMiss_F'] = dk.groupby(['Season', 'GD'])['MMiss_F'].transform('mean')
dk['TPenalties_F'] = dk.groupby(['Season', 'GD'])['MPenalties_F'].transform('mean')
dk['TShots_F'] = dk.groupby(['Season', 'GD'])['MShots_F'].transform('mean')
dk['TTakeaways_F'] = dk.groupby(['Season', 'GD'])['MTakeaways_F'].transform('mean')
dk['TBlocks_A'] = dk.groupby(['Season', 'GD'])['MBlocks_A'].transform('mean')
dk['TFaceoffs_A'] = dk.groupby(['Season', 'GD'])['MFaceoffs_A'].transform('mean')
dk['TGiveaways_A'] = dk.groupby(['Season', 'GD'])['MGiveaways_A'].transform('mean')
dk['TGoals_A'] = dk.groupby(['Season', 'GD'])['MGoals_A'].transform('mean')
dk['THits_A'] = dk.groupby(['Season', 'GD'])['MHits_A'].transform('mean')
dk['TMiss_A'] = dk.groupby(['Season', 'GD'])['MMiss_A'].transform('mean')
dk['TPenalties_A'] = dk.groupby(['Season', 'GD'])['MPenalties_A'].transform('mean')
dk['TShots_A'] = dk.groupby(['Season', 'GD'])['MShots_A'].transform('mean')
dk['TTakeaways_A'] = dk.groupby(['Season', 'GD'])['MTakeaways_A'].transform('mean')
dk.head()

,Season,TeamCode,GD,MBlocks_F,MFaceoffs_F,MGiveaways_F,MGoals_F,MHits_F,MMiss_F,MPenalties_F,MShots_F,MTakeaways_F,MBlocks_A,MFaceoffs_A,MGiveaways_A,MGoals_A,MHits_A,MMiss_A,MPenalties_A,MShots_A,MTakeaways_A,TBlocks_F,TFaceoffs_F,TGiveaways_F,TGoals_F,THits_F,TMiss_F,TPenalties_F,TShots_F,TTakeaways_F,TBlocks_A,TFaceoffs_A,TGiveaways_A,TGoals_A,THits_A,TMiss_A,TPenalties_A,TShots_A,TTakeaways_A
0,2010,MTL,-1.0,1.784810,2.822785,1.088608,0.367089,2.784810,1.291139,0.544304,2.696203,0.544304,1.582278,2.810127,1.240506,0.443038,3.113924,1.481013,0.620253,3.202532,1.050633,1.374483,2.516850,0.681846,0.366420,2.413534,1.065690,0.445790,2.591455,0.603642,1.327431,2.686208,0.955630,0.394511,2.802983,1.121055,0.432412,2.708201,0.809985
1,2010,TOR,1.0,1.488372,2.244186,0.837209,0.383721,2.476744,1.034884,0.430233,2.244186,0.604651,1.476744,2.441860,1.209302,0.406977,2.383721,1.151163,0.337209,2.279070,0.697674,1.357536,2.493580,0.668672,0.368602,2.358437,1.060113,0.444503,2.555321,0.605353,1.309502,2.643892,0.937125,0.395125,2.732869,1.097155,0.427853,2.666692,0.807829
32,2010,MTL,-2.0,1.437500,3.166667,0.895833,0.375000,2.562500,1.041667,0.562500,2.937500,0.375000,1.416667,3.020833,1.166667,0.395833,2.812500,1.125000,0.750000,2.666667,0.791667,1.240119,2.398981,0.578468,0.370569,2.048669,0.987817,0.471869,2.341516,0.509553,1.197821,2.487231,0.826497,0.379039,2.341475,1.043890,0.420119,2.492155,0.750887
34,2010,TOR,2.0,1.333333,2.357143,0.952381,0.428571,2.309524,1.190476,0.476190,2.238095,0.571429,1.880952,2.642857,1.285714,0.452381,2.500000,1.214286,0.523810,2.595238,1.023810,1.211139,2.339097,0.573371,0.364781,2.021929,0.970485,0.444882,2.294571,0.498110,1.200102,2.459261,0.812138,0.390298,2.330835,1.026941,0.421265,2.427477,0.759147
77,2010,PIT,-1.0,1.202899,2.594203,0.449275,0.347826,3.000000,0.855072,0.507246,2.507246,0.492754,1.347826,2.521739,0.666667,0.463768,3.333333,0.956522,0.449275,2.579710,0.449275,1.374483,2.516850,0.681846,0.366420,2.413534,1.065690,0.445790,2.591455,0.603642,1.327431,2.686208,0.955630,0.394511,2.802983,1.121055,0.432412,2.708201,0.809985


In [113]:
dk = dk[['GD', 'TBlocks_F', 'TFaceoffs_F', 'TGiveaways_F', 'TGoals_F', 'THits_F', 'TMiss_F', 'TPenalties_F', 'TShots_F', 'TTakeaways_F', 'TBlocks_A', 'TFaceoffs_A', 'TGiveaways_A', 'TGoals_A', 'THits_A', 'TMiss_A', 'TPenalties_A', 'TShots_A', 'TTakeaways_A']]
dk = dk.drop_duplicates(['GD'])
dk = dk.sort_values(['GD'], ascending=[False])
dk.set_index(['GD'])

,TBlocks_F,TFaceoffs_F,TGiveaways_F,TGoals_F,THits_F,TMiss_F,TPenalties_F,TShots_F,TTakeaways_F,TBlocks_A,TFaceoffs_A,TGiveaways_A,TGoals_A,THits_A,TMiss_A,TPenalties_A,TShots_A,TTakeaways_A
GD,,,,,,,,,,,,,,,,,,
8.0,4.000000,3.000000,0.000000,0.000000,1.000000,2.000000,1.000000,5.000000,0.000000,0.000000,6.000000,0.000000,0.000000,2.000000,2.000000,1.000000,4.000000,0.000000
7.0,1.227273,2.515152,0.409091,0.454545,1.318182,0.575758,0.439394,1.818182,0.484848,1.015152,2.151515,0.151515,0.409091,1.818182,0.878788,0.272727,3.060606,0.803030
6.0,0.726190,2.238889,0.561905,0.315873,1.268254,1.202381,0.491270,1.715873,0.290476,0.772222,2.222222,0.359524,0.464286,1.389683,0.650794,0.612698,1.830952,0.450794
5.0,0.768008,1.992912,0.334866,0.279502,1.599617,0.561111,0.679885,1.902107,0.410153,0.814943,2.135057,0.448851,0.472031,1.593487,0.715134,0.486207,1.830843,0.473180
4.0,0.863633,1.951981,0.400111,0.280763,1.894409,0.922831,0.643248,1.876572,0.455956,1.065953,2.186687,0.618993,0.467808,1.758716,0.707163,0.611978,1.980337,0.608767
3.0,1.247149,2.431223,0.584393,0.336082,1.960044,0.985400,0.481207,2.388895,0.548665,1.203270,2.562140,0.786626,0.385784,2.038216,1.028983,0.404051,2.378104,0.749588
2.0,1.211139,2.339097,0.573371,0.364781,2.021929,0.970485,0.444882,2.294571,0.498110,1.200102,2.459261,0.812138,0.390298,2.330835,1.026941,0.421265,2.427477,0.759147
1.0,1.357536,2.493580,0.668672,0.368602,2.358437,1.060113,0.444503,2.555321,0.605353,1.309502,2.643892,0.937125,0.395125,2.732869,1.097155,0.427853,2.666692,0.807829
0.0,1.175859,2.249581,0.524132,0.380943,1.960600,0.918994,0.397964,2.154843,0.513907,1.119420,2.336651,0.719260,0.343014,2.270362,0.945199,0.405949,2.291306,0.696743


In [114]:
beginningtex = """\\documentclass{report}
\\usepackage{booktabs}
\\begin{document}"""
endtex = "\end{document}"

f = open('/Users/stefanostselios/Brock University/Kevin Mongeon - StephanosShare/out/latex/events/gd_even_strength_events.tex', 'w')
f.write(beginningtex)
f.write(dk.to_latex())
f.write(endtex)
f.close()